In [ ]:
!nvidia-smi

Tue Oct 17 09:19:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## 모델 로딩

4bit 양자화

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

# 데이터 로딩

In [ ]:
from datasets import load_dataset

data = load_dataset("json",
                    data_files={"train":"test_공인회계사법.json"})
data

In [ ]:
[row for row in data['train']]

In [ ]:
data = data.map(
    lambda x: {
        'text':
        f"""User: {x['법령명']}

Assistant: {x['전문']}<|endoftext|>"""
        }
) #instruction, input, output

In [ ]:
[row['text'] for row in data['train']]

## 텍스트 데이터 tokenize

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
data["train"][0]["text"]

'User: 공인회계사법\n\nAssistant: {"제1장 총칙": {"제1조": " 이 법은 공인회계사제도를 확립함으로써 국민의 권익보호와 기업의 건전한 경영 및 국가경제의 발전에 이바지함을 목적으로 한다.", "제2조": "1. 회계에 관한 감사ㆍ감정ㆍ증명ㆍ계산ㆍ정리ㆍ입안 또는 법인설립등에 관한 회계", "제3조": " 공인회계사시험에 합격한 자는 공인회계사의 자격이 있다.", "제4조": "1. 미성년자 또는 피성년후견인"}, "제2장 시험": {"제5조(공인회계사시험)": "①공인회계사시험(이하 "시험"이라 한다)은 금융위원회가 실시하되, 제1차시험과 제2차시험으로 이루어진다. <개정 2001.3.28, 2003.12.11, 2008.2.29>", "제5조의2(부정행위자에 대한 제재)": "① 금융위원회는 다음 각 호의 어느 하나에 해당하는 사람에 대하여는 해당 시험을 정지시키거나 합격 결정을 취소한다.", "제6조(시험의 일부면제)": "①다음 각호의 1에 해당하는 자에 대하여는 시험중 제1차시험을 면제한다. <개정 1998.1.13, 2003.12.11, 2005.7.29, 2005.12.29, 2010.5.17, 2011.7.21>", "제6조의2(공인회계사자격제도심의위원회)": "①공인회계사자격의 취득과 관련한 다음 각호의 사항을 심의하기 위하여 금융위원회 소속하에 공인회계사자격제도심의위원회를 둔다. <개정 2008.2.29>"}}<|endoftext|>'

In [ ]:
tokenized_text = tokenizer(data["train"][0]["text"])
tokenized_text

{'input_ids': [56, 426, 85, 29, 7274, 6841, 7769, 202, 202, 36, 12732, 15097, 15741, 29, 224, 94, 5, 400, 20, 349, 888, 2088, 5, 29, 224, 94, 5, 400, 20, 510, 5, 29, 490, 307, 1008, 296, 7274, 6841, 347, 3045, 301, 9233, 953, 2848, 1182, 285, 13204, 3611, 441, 1035, 285, 7959, 288, 1851, 903, 1278, 1566, 285, 1775, 274, 14250, 953, 276, 2868, 339, 873, 9098, 15, 490, 400, 21, 510, 5, 29, 490, 20, 17, 5896, 274, 2708, 2031, 1091, 13471, 1091, 13389, 1091, 19027, 1091, 12374, 1091, 664, 532, 2259, 3948, 15034, 968, 274, 2708, 5896, 9850, 490, 400, 22, 510, 5, 29, 490, 7274, 6841, 347, 3726, 274, 4333, 288, 454, 272, 7274, 6841, 347, 285, 3445, 270, 327, 267, 9098, 15, 490, 400, 23, 510, 5, 29, 490, 20, 17, 17256, 2259, 881, 12719, 694, 27911, 5, 96, 15, 490, 400, 21, 349, 2888, 5, 29, 224, 94, 5, 400, 24, 510, 11, 5108, 6841, 347, 3726, 12, 5, 29, 490, 18753, 258, 5108, 6841, 347, 3726, 11, 4148, 490, 3726, 5, 7194, 873, 12, 296, 1354, 1331, 293, 1768, 284, 605, 15, 480, 20, 609, 3726, 3

In [ ]:
token_text = [tokenizer.decode(id) for id in tokenized_text["input_ids"]]
", ".join(token_text)

'U, se, r, :,  공인, 회계, 사법, \n, \n, A, ss, ist, ant, :,  , {, ", 제, 1, 장,  총, 칙, ", :,  , {, ", 제, 1, 조, ", :,  ",  이,  법, 은,  공인, 회계, 사, 제도, 를,  확립, 함, 으로써,  국민, 의,  권익, 보호, 와,  기업, 의,  건전, 한,  경영,  및,  국가, 경제, 의,  발전, 에,  이바지, 함, 을,  목적, 으로,  한다, .", ,,  ", 제, 2, 조, ", :,  ", 1, .,  회계, 에,  관한,  감사, ㆍ, 감정, ㆍ, 증명, ㆍ, 계산, ㆍ, 정리, ㆍ, 입, 안,  또는,  법인, 설립, 등, 에,  관한,  회계, ",,  ", 제, 3, 조, ", :,  ",  공인, 회계, 사, 시험, 에,  합격, 한,  자, 는,  공인, 회계, 사, 의,  자격, 이,  있, 다, .", ,,  ", 제, 4, 조, ", :,  ", 1, .,  미성년자,  또는,  피, 성년, 후, 견인, ", }, ,,  ", 제, 2, 장,  시험, ", :,  , {, ", 제, 5, 조, (, 공인, 회계, 사, 시험, ), ", :,  ", �, �, 공인, 회계, 사, 시험, (, 이하,  ", 시험, ", 이라,  한다, ), 은,  금융, 위원회, 가,  실시, 하, 되, ,,  제, 1, 차, 시험, 과,  제, 2, 차, 시험, 으로,  이루어진다, .,  <, 개정,  2001, ., 3, ., 28, ,,  2003, ., 12, ., 11, ,,  2008, ., 2, ., 29, >, ",,  ", 제, 5, 조, 의, 2, (, 부정, 행위, 자, 에,  대한,  제재, ), ", :,  ", �, �,  금융, 위원회, 는,  다음,  각,  호의,  어느,  하나, 에,  해당, 하, 는,  사람, 에,  대하, 여, 는,  해당,  시험, 을,  정지, 시키, 거나,  합격,  결정, 을,  취소, 한다, .",

## PEFT 학습 준비

In [ ]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
# 학습되는 파라미터 수 출력
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"""trainable params: {trainable_params}
all params: {all_param}
trainable%: {100 * trainable_params / all_param}""")

trainable params: 6553600
all params: 6608701440
trainable%: 0.09916622894073424


## 학습하기

In [ ]:
import transformers
import gc
import torch

# gc.collect()
# torch.cuda.empty_cache()


tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        max_steps=50, # 100 step 학습시, V100 기준 약 2분 소요
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        learning_rate=1e-4,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        logging_steps=10,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
trainer.train()

# logging_steps = 10
# learning_rate = 2e-4

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.935200
20,0.475800
30,0.198900
40,0.112600
50,0.086700


TrainOutput(global_step=50, training_loss=0.36183539032936096, metrics={'train_runtime': 232.897, 'train_samples_per_second': 0.429, 'train_steps_per_second': 0.215, 'total_flos': 1586895406080000.0, 'train_loss': 0.36183539032936096, 'epoch': 50.0})

In [ ]:
# !pip install GPUtil

# from GPUtil import showUtilization as gpu_usage
# gpu_usage()

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=9fa1d40eef312641737929296b4d546134586a75d76013514c5228cf168cd5b9
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% | 69% |


In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)

<weakproxy at 0x7a56f3e2fce0 to Device at 0x7a56f3e41d20>

In [ ]:
model.eval()
model.config.use_cache = True

In [ ]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 256

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg,
    )

    return tokenizer.decode(gened[0])

In [ ]:
# eos_token_id=2,

In [ ]:
gen('공인회계사법')

torch.Size([1, 14])
no pkv
torch.Size([1, 14])
torch.Size([1, 15])
torch.Size([1, 40, 14, 128])
torch.Size([1, 1])
torch.Size([1, 16])
torch.Size([1, 40, 15, 128])
torch.Size([1, 1])
torch.Size([1, 17])
torch.Size([1, 40, 16, 128])
torch.Size([1, 1])
torch.Size([1, 18])
torch.Size([1, 40, 17, 128])
torch.Size([1, 1])
torch.Size([1, 19])
torch.Size([1, 40, 18, 128])
torch.Size([1, 1])
torch.Size([1, 20])
torch.Size([1, 40, 19, 128])
torch.Size([1, 1])
torch.Size([1, 21])
torch.Size([1, 40, 20, 128])
torch.Size([1, 1])
torch.Size([1, 22])
torch.Size([1, 40, 21, 128])
torch.Size([1, 1])
torch.Size([1, 23])
torch.Size([1, 40, 22, 128])
torch.Size([1, 1])
torch.Size([1, 24])
torch.Size([1, 40, 23, 128])
torch.Size([1, 1])
torch.Size([1, 25])
torch.Size([1, 40, 24, 128])
torch.Size([1, 1])
torch.Size([1, 26])
torch.Size([1, 40, 25, 128])
torch.Size([1, 1])
torch.Size([1, 27])
torch.Size([1, 40, 26, 128])
torch.Size([1, 1])
torch.Size([1, 28])
torch.Size([1, 40, 27, 128])
torch.Size([1, 1])
t

'User: 공인회계사법\n\nAssistant: {"제1장 총칙": {"제1조": " 이 법은 공인회계사제도를 확립함으로써 국민의 권익보호와 기업의 건전한 경영 및 국가경제의 발전에 이바지함을 목적으로 한다.", "제2조": "1. 회계에 관한 감사ㆍ감정ㆍ증명ㆍ계산ㆍ정리ㆍ입안 또는 법인설립등에 관한 회계", "제3조": " 공인회계사시험에 합격한 자는 공인회계사의 자격이 있다.", "제4조": "1. 미성년자 또는 피성년후견인"}, "제2장 시험": {"제5조(공인회계사시험)": "①공인회계사시험(이하 "시험"이라 한다)은 금융위원회가 실시하되, 제1차시험과 제2차시험으로 이루어진다. <개정 2001.3.28, 2003.12.11, 2008.2.29>", "제5조의2(부정행위자에 대한 제재)": "① 금융위원회는 다음 각 호의 어느 하나에 해당하는 사람에 대하여는 해당 시험을 정지시키거나 합격 결정을 취소한다.",'

endoftext 이 안나왔는데 나오려면 1000개 이상의 데이터셋을 학습시켜야 함

## 모델 저장 & 업로드

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub('julietz/koalpaca-polyglot-12.8b-law_248923')

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/julietz/koalpaca-polyglot-12.8b-law_248923/commit/c9064bc6e3ad6988e114566aa42b23b97b472ab9', commit_message='Upload model', commit_description='', oid='c9064bc6e3ad6988e114566aa42b23b97b472ab9', pr_url=None, pr_revision=None, pr_num=None)

Reference: [KoAlpaca QLoRA Training Original Notebook by Beomi](https://colab.research.google.com/gist/Beomi/a3032e4eaa33b86fdf8de1f47f15a647/2023_05_26_bnb_4bit_koalpaca_v1_1a_on_polyglot_ko_12_8b.ipynb)